### Picking which words are valid for the crossword
The words will be saved in a csv file. alongside any relevant information.

In [307]:
import os
import re
import numpy as np
import pandas as pd
import time
import json

frequency_path = os.path.join(os.getcwd(),'..','data','korpus','korpus_frequency.csv')
wordforms_path = os.path.join(os.getcwd(),'..','data','gabra','wordforms.csv')
lessiku_path = os.path.join(os.getcwd(),'..','data','gabra','lexemes.csv')
final_lexemes_path = os.path.join(os.getcwd(),'..','data','gabra','final','lexemes.csv')


In [308]:
#Load frequency counts of words from korpus
df_frequency = pd.read_csv(frequency_path)

#Load lexemes
df_lexemes = pd.read_csv(lessiku_path, encoding='utf-32')[["_id","lemma","root","sources","phonetic","pos","glosses"]]

In [309]:
df_lexemes = df_lexemes[df_lexemes['sources'] != "['UserFeedback']"]
df_lexemes = df_lexemes.dropna()

In [310]:
df_lexemes

,_id,lemma,root,sources,phonetic,pos,glosses
0,5200a366e36f237975000783,badbad,{'radicals': 'b-d-b-d'},"['Spagnol2011', 'Falzon2013']",bɐdbɐt,VERB,"[{'gloss': 'to fornicate', 'examples': []}, {'..."
1,5200a366e36f237975000784,tbadbad,{'radicals': 'b-d-b-d'},['Spagnol2011'],dbɐdbɐt,VERB,[{'gloss': 'to act in a rough or anti-social m...
2,5200a366e36f237975000785,bagħbas,{'radicals': 'b-għ-b-s'},"['Spagnol2011', 'Falzon2013']",bɐːbɐs,VERB,"[{'gloss': 'to touch', 'examples': []}, {'glos..."
3,5200a366e36f237975000786,tbagħbas,{'radicals': 'b-għ-b-s'},['Spagnol2011'],dbɐːbɐs,VERB,[{'gloss': 'to be damaged as a result of handl...
4,5200a366e36f237975000787,bagħtar,{'radicals': 'b-għ-t-r'},['Spagnol2011'],bɐːtɐr,VERB,"[{'gloss': 'to wade through mud', 'examples': ..."
...,...,...,...,...,...,...,...
16833,569002ffdff0600b5ab1434e,mgħarwel,{'radicals': 'għ-r-w-l'},"['UserFeedback', 'DM2015']",mɐːrwɛl,ADJ,"[{'gloss': 'swarming', 'examples': []}]"
16837,56966190189d1d6376783b94,ħaxixa,{'radicals': 'ħ-x-x'},['DM2015'],hɐʃɪʃɐ,NOUN,"[{'gloss': 'colloquial term for marijuana', 'e..."
16846,569794e16e68020d08030c4d,kitba,{'radicals': 'k-t-b'},['DM2015'],kɪdbɐ,NOUN,"[{'gloss': 'writing', 'examples': [{'example':..."
16982,5714de59418362c403b9c3ff,xagħat,{'radicals': 'x-għ-t'},['DM2015'],ʃɐːt,NOUN,"[{'gloss': 'caterpillar', 'examples': [{'examp..."


In [311]:
print(df_lexemes[df_lexemes['_id'] =='5200a368e36f23798800000f'].glosses.values[0])
# print(df_lexemes.iloc[3885])

[{'gloss': 'craft', 'examples': []}, {'gloss': 'skill', 'examples': [{'example': 'L- għalliem tal- woodwork , ċertu Mr Gauci , kien jagħmel ħiltu kollha biex lil dawn it- tfal iqabbadhom is-sengħa ta’ mastrudaxxa .', 'type': 'full'}]}, {'gloss': 'ability', 'examples': []}]


##### Parsing glosses and seperating examples

In [312]:
glosses = []
examples = []

for txt in df_lexemes['glosses']:

    result = re.findall('{(.*?)}', txt)
    
    gloss   = [i.split(',')[0] for i in result]
    gloss   = ', '.join([g[10:-1] for g in gloss])
    glosses.append(gloss)
    
#     if len(result[0].split(',')) == 2:
#         example = [i.split(',')[1] for i in result]
#         example = ', '.join([i[14:-1] for i in example])
#         print(example)
#         examples.append(example)
#     else:
#         examples.append('')

df_lexemes['glosses'] = glosses

In [313]:
df_lexemes

,_id,lemma,root,sources,phonetic,pos,glosses
0,5200a366e36f237975000783,badbad,{'radicals': 'b-d-b-d'},"['Spagnol2011', 'Falzon2013']",bɐdbɐt,VERB,"to fornicate, to cough a lot"
1,5200a366e36f237975000784,tbadbad,{'radicals': 'b-d-b-d'},['Spagnol2011'],dbɐdbɐt,VERB,to act in a rough or anti-social manner
2,5200a366e36f237975000785,bagħbas,{'radicals': 'b-għ-b-s'},"['Spagnol2011', 'Falzon2013']",bɐːbɐs,VERB,"to touch, to fondle"
3,5200a366e36f237975000786,tbagħbas,{'radicals': 'b-għ-b-s'},['Spagnol2011'],dbɐːbɐs,VERB,"to be damaged as a result of handling, to be m..."
4,5200a366e36f237975000787,bagħtar,{'radicals': 'b-għ-t-r'},['Spagnol2011'],bɐːtɐr,VERB,to wade through mud
...,...,...,...,...,...,...,...
16833,569002ffdff0600b5ab1434e,mgħarwel,{'radicals': 'għ-r-w-l'},"['UserFeedback', 'DM2015']",mɐːrwɛl,ADJ,swarming
16837,56966190189d1d6376783b94,ħaxixa,{'radicals': 'ħ-x-x'},['DM2015'],hɐʃɪʃɐ,NOUN,colloquial term for marijuana
16846,569794e16e68020d08030c4d,kitba,{'radicals': 'k-t-b'},['DM2015'],kɪdbɐ,NOUN,"writing, the act of writing, written text, cre..."
16982,5714de59418362c403b9c3ff,xagħat,{'radicals': 'x-għ-t'},['DM2015'],ʃɐːt,NOUN,"caterpillar, ""Ix- xagħat li jfaqqas mill- baj..."


In [314]:
df_lexemes

,_id,lemma,root,sources,phonetic,pos,glosses
0,5200a366e36f237975000783,badbad,{'radicals': 'b-d-b-d'},"['Spagnol2011', 'Falzon2013']",bɐdbɐt,VERB,"to fornicate, to cough a lot"
1,5200a366e36f237975000784,tbadbad,{'radicals': 'b-d-b-d'},['Spagnol2011'],dbɐdbɐt,VERB,to act in a rough or anti-social manner
2,5200a366e36f237975000785,bagħbas,{'radicals': 'b-għ-b-s'},"['Spagnol2011', 'Falzon2013']",bɐːbɐs,VERB,"to touch, to fondle"
3,5200a366e36f237975000786,tbagħbas,{'radicals': 'b-għ-b-s'},['Spagnol2011'],dbɐːbɐs,VERB,"to be damaged as a result of handling, to be m..."
4,5200a366e36f237975000787,bagħtar,{'radicals': 'b-għ-t-r'},['Spagnol2011'],bɐːtɐr,VERB,to wade through mud
...,...,...,...,...,...,...,...
16833,569002ffdff0600b5ab1434e,mgħarwel,{'radicals': 'għ-r-w-l'},"['UserFeedback', 'DM2015']",mɐːrwɛl,ADJ,swarming
16837,56966190189d1d6376783b94,ħaxixa,{'radicals': 'ħ-x-x'},['DM2015'],hɐʃɪʃɐ,NOUN,colloquial term for marijuana
16846,569794e16e68020d08030c4d,kitba,{'radicals': 'k-t-b'},['DM2015'],kɪdbɐ,NOUN,"writing, the act of writing, written text, cre..."
16982,5714de59418362c403b9c3ff,xagħat,{'radicals': 'x-għ-t'},['DM2015'],ʃɐːt,NOUN,"caterpillar, ""Ix- xagħat li jfaqqas mill- baj..."


### Parsing roots

In [315]:
regex = re.compile('[a-zġħċż][a-zġħċż]?-[a-zġħċż][a-zġħċż]?-[a-zġħċż][a-zġħċż]?-?[a-zġħċż]?[a-zġħċż]?',)

regex.findall('b-d-b-t')[0]
df_lexemes['root'] = df_lexemes['root'].apply(lambda s: ''.join(regex.findall(s)))
df_lexemes

,_id,lemma,root,sources,phonetic,pos,glosses
0,5200a366e36f237975000783,badbad,b-d-b-d,"['Spagnol2011', 'Falzon2013']",bɐdbɐt,VERB,"to fornicate, to cough a lot"
1,5200a366e36f237975000784,tbadbad,b-d-b-d,['Spagnol2011'],dbɐdbɐt,VERB,to act in a rough or anti-social manner
2,5200a366e36f237975000785,bagħbas,b-għ-b-s,"['Spagnol2011', 'Falzon2013']",bɐːbɐs,VERB,"to touch, to fondle"
3,5200a366e36f237975000786,tbagħbas,b-għ-b-s,['Spagnol2011'],dbɐːbɐs,VERB,"to be damaged as a result of handling, to be m..."
4,5200a366e36f237975000787,bagħtar,b-għ-t-r,['Spagnol2011'],bɐːtɐr,VERB,to wade through mud
...,...,...,...,...,...,...,...
16833,569002ffdff0600b5ab1434e,mgħarwel,għ-r-w-l,"['UserFeedback', 'DM2015']",mɐːrwɛl,ADJ,swarming
16837,56966190189d1d6376783b94,ħaxixa,ħ-x-x,['DM2015'],hɐʃɪʃɐ,NOUN,colloquial term for marijuana
16846,569794e16e68020d08030c4d,kitba,k-t-b,['DM2015'],kɪdbɐ,NOUN,"writing, the act of writing, written text, cre..."
16982,5714de59418362c403b9c3ff,xagħat,x-għ-t,['DM2015'],ʃɐːt,NOUN,"caterpillar, ""Ix- xagħat li jfaqqas mill- baj..."


### Replacing abbreviations

In [316]:
df_lexemes[df_lexemes['glosses'].str.contains('sth.')]

,_id,lemma,root,sources,phonetic,pos,glosses
42,5200a366e36f2379750007ae,bergħen,b-r-għ-n,['Spagnol2011'],bɛrɛːn,VERB,"to mull sth. over, to make sb. angry"
58,5200a366e36f2379750007be,bexlek,b-x-l-k,['Spagnol2011'],bɛʃlɛk,VERB,to occupy one's mind with sth.
61,5200a366e36f2379750007c2,bażwar,b-ż-w-r,['Spagnol2011'],bɐzwɐr,VERB,"to exhaust sb., to perform a task clumsily, to..."
66,5200a366e36f2379750007c7,tbeżbeż,b-ż-b-ż,['Spagnol2011'],dbɛzbɛs,VERB,"to be reprimanded, to be castigated, to experi..."
76,5200a366e36f2379750007d1,ċeflaq,ċ-f-l-q,['Spagnol2011'],ʧɛflɐʔ,VERB,to shift sth.
...,...,...,...,...,...,...,...
16755,56802c9b5afadbfd19202677,xemnieq,x-m-n-q,['DM2015'],ʃɛmnɪːʔ,NOUN,sb. who depreciates or displays contempt for sth.
16772,568034c35afadbfd19202722,xaqliebi,x-q-l-b,['DM2015'],ʃɐʔlɪːbɪ,ADJ,"tending to topple things over, having a pencha..."
16792,568047995afadbfd192027fc,żembieħ,ż-m-b-ħ,['DM2015'],zɛmbɪːh,NOUN,"sth. that causes swelling, sb. who tends to fl..."
16822,568075035afadbfd19204436,żerżieq,ż-r-ż-q,['DM2015'],zɛrzɪːʔ,NOUN,sth. that enables things to slide down


In [317]:
df_lexemes['glosses'] = df_lexemes['glosses'].apply(lambda s: s.replace('sb.','somebody')
                                                    .replace('sth.','something')
                                                    .replace('sb','somebody')
                                                    .replace('sth','something'))

df_lexemes[df_lexemes['glosses'].str.contains('somebody')]

,_id,lemma,root,sources,phonetic,pos,glosses
6,5200a366e36f237975000789,bahrad,b-h-r-d,['Spagnol2011'],bɐːrɐt,VERB,to make somebody indulge in enjoyment or frivo...
21,5200a366e36f237975000799,bambal,b-m-b-l,['Spagnol2011'],bɐmbɐl,VERB,"to hit somebody repeatedly, to punish, to pummel"
30,5200a366e36f2379750007a2,baqbaq,b-q-b-q,['Spagnol2011'],bɐʔbɐʔ,VERB,"to bubble, to gurgle, to boil, to get somebody..."
42,5200a366e36f2379750007ae,bergħen,b-r-għ-n,['Spagnol2011'],bɛrɛːn,VERB,"to mull something over, to make somebody angry"
59,5200a366e36f2379750007bf,bixkel,b-x-k-l,"['Spagnol2011', 'Falzon2013']",bɪʃkɛl,VERB,"to entwine, to make wicker, to involve somebod..."
...,...,...,...,...,...,...,...
16803,5680665b5afadbfd19203a67,żekżiek,ż-k-ż-k,['DM2015'],zɛgzɪːk,NOUN,somebody who tends to be rude
16809,568068e65afadbfd19204392,żambari,ż-m-b-r,['DM2015'],zɐmbɐrɪ,ADJ,"somebody who drinks a lot, a lush"
16812,56806ba15afadbfd192043b2,żengiel,ż-n-g-l,['DM2015'],zɛngɪːl,NOUN,somebody who rocks a child in a cradle
16819,568072005afadbfd19204406,żarmatur,ż-r-m-j,['DM2015'],zɐrmɐtʊr,NOUN,somebody who dismantles


In [318]:
df_lexemes = df_lexemes.reset_index(drop=True)
df_lexemes.to_csv(final_lexemes_path, index=False)